# A numerical model of the two-dimensional viscid Burgers' equations

This notebook contains a [GT4Py](https://github.com/eth-cscs/gridtools4py.git)-powered implementation of a finite difference-based numerical method for the two-dimensional viscid Burgers' equations

\begin{equation*}
    \begin{aligned}
        & \dfrac{\partial u}{\partial t} + u \dfrac{\partial u}{\partial x} + v \dfrac{\partial u}{\partial y} = \mu \left( \dfrac{\partial^2 u}{\partial x^2} + \dfrac{\partial^2 u}{\partial y^2} \right) \, , \\
        & \dfrac{\partial v}{\partial t} + u \dfrac{\partial v}{\partial x} + v \dfrac{\partial v}{\partial y} = \mu \left( \dfrac{\partial^2 v}{\partial x^2} + \dfrac{\partial^2 v}{\partial y^2} \right) \, .
    \end{aligned}
\end{equation*}

Here, $\mathbf{v} = (u, \, v)$ is a velocity field and $\mu$ is the diffusion coefficient. The discretization is set over a regular Cartesian grid and couples a fifth-order upwind advection scheme (Baldauf, 2008) with fourth-order centered formulas to calculate the second-order spatial derivatives. Time stepping is performed via the third-order Runge-Kutta scheme proposed by Wicker and Skamarock (2002). The following numerical examples are available:

* `use_case = "zhao"`: initial-boundary value problem discussed in Zhao et al. (2011) and whose analytical solution is

\begin{equation*}
    \begin{aligned}
        u(x, y, t) & = - 2 \mu \dfrac{2 \pi \, \text{e}^{-5 \pi^2 \mu t} \cos(2 \pi x) \sin(\pi y)}{2 + \text{e}^{-5 \pi^2 \mu t} \sin(2 \pi x) \sin(\pi y)} \, , \\
        v(x, y, t) & = - 2 \mu \dfrac{\pi \, \text{e}^{-5 \pi^2 \mu t} \sin(2 \pi x) \cos(\pi y)}{2 + \text{e}^{-5 \pi^2 \mu t} \sin(2 \pi x) \sin(\pi y)} \, ;
    \end{aligned}
\end{equation*}


* `use_case = "hopf_cole"`: the Hopf-Cole test case (Zhu et al., 2010), which admits the exact solution

\begin{equation*}
    \begin{aligned}
        u(x, y, t) & = \dfrac{3}{4} - \dfrac{1}{4 \left( 1 + \text{e}^{(- t - 4x + 4y)/32 \mu} \right)} \, , \\
        v(x, y, t) & = \dfrac{3}{4} + \dfrac{1}{4 \left( 1 + \text{e}^{(- t - 4x + 4y)/32 \mu} \right)} \, .
    \end{aligned}
\end{equation*}

Imports and customizable settings
--

In [6]:
import gt4py as gt
import numpy as np
import time

# gt4py
backend = "gtx86"  # "debug", "numpy", "gtx86", "gtmc", "gtcuda"
dtype = np.float64
halo = (3, 3, 0)
rebuild = False

# use case
use_case = "zhao"  # "zhao", "hopf_cole"

# diffusion coefficient
mu = 0.1

# grid
factor = 8
nx = 10 * 2**factor + 1
ny = 10 * 2**factor + 1

# time
cfl = 1.
timestep = cfl / (nx-1)**2
niter = 5

# output
print_period = -1

Subroutines
--

In [2]:
@gt.gtscript.function
def absolute_value(phi):
    abs_phi = phi if phi > 0. else -phi
    return abs_phi


@gt.gtscript.function
def advection_x(dx, u, abs_u, phi):
    adv_phi_x = u[0, 0, 0] / (60. * dx) * (
        + 45. * (phi[1, 0, 0] - phi[-1, 0, 0])
        -  9. * (phi[2, 0, 0] - phi[-2, 0, 0])
        +       (phi[3, 0, 0] - phi[-3, 0, 0])
    ) - abs_u[0, 0, 0] / (60. * dx) * (
        +       (phi[3, 0, 0] + phi[-3, 0, 0])
        -  6. * (phi[2, 0, 0] + phi[-2, 0, 0])
        + 15. * (phi[1, 0, 0] + phi[-1, 0, 0])
        - 20. * phi[0, 0, 0]
    )
    return adv_phi_x


@gt.gtscript.function
def advection_y(dy, v, abs_v, phi):
    adv_phi_y = v[0, 0, 0] / (60. * dy) * (
        + 45. * (phi[0, 1, 0] - phi[0, -1, 0])
        -  9. * (phi[0, 2, 0] - phi[0, -2, 0])
        +       (phi[0, 3, 0] - phi[0, -3, 0])
    ) - abs_v[0, 0, 0] / (60. * dy) * (
        +       (phi[0, 3, 0] + phi[0, -3, 0])
        -  6. * (phi[0, 2, 0] + phi[0, -2, 0])
        + 15. * (phi[0, 1, 0] + phi[0, -1, 0])
        - 20. * phi[0, 0, 0]
    )
    return adv_phi_y


@gt.gtscript.function
def advection(dx, dy, u, v):
    abs_u = absolute_value(phi=u)
    abs_v = absolute_value(phi=v)
    
    adv_u_x = advection_x(dx=dx, u=u, abs_u=abs_u, phi=u)
    adv_u_y = advection_y(dy=dy, v=v, abs_v=abs_v, phi=u)
    adv_u = adv_u_x[0, 0, 0] + adv_u_y[0, 0, 0]
    
    adv_v_x = advection_x(dx=dx, u=u, abs_u=abs_u, phi=v)
    adv_v_y = advection_y(dy=dy, v=v, abs_v=abs_v, phi=v)
    adv_v = adv_v_x[0, 0, 0] + adv_v_y[0, 0, 0]
    
    return adv_u, adv_v


@gt.gtscript.function
def diffusion_x(dx, phi):
    diff_phi = (
        -       phi[-2, 0, 0]
        + 16. * phi[-1, 0, 0]
        - 30. * phi[0, 0, 0]
        + 16. * phi[1, 0, 0]
        -       phi[2, 0, 0]
    ) / (12. * dx**2)
    return diff_phi


@gt.gtscript.function
def diffusion_y(dy, phi):
    diff_phi = (
        -       phi[0, -2, 0]
        + 16. * phi[0, -1, 0]
        - 30. * phi[0, 0, 0]
        + 16. * phi[0, 1, 0]
        -       phi[0, 2, 0]
    ) / (12. * dy**2)
    return diff_phi


@gt.gtscript.function
def diffusion(dx, dy, u, v):
    diff_u_x = diffusion_x(dx=dx, phi=u)
    diff_u_y = diffusion_y(dy=dy, phi=u)
    diff_u = diff_u_x[0, 0, 0] + diff_u_y[0, 0, 0]
    
    diff_v_x = diffusion_x(dx=dx, phi=v)
    diff_v_y = diffusion_y(dy=dy, phi=v)
    diff_v = diff_v_x[0, 0, 0] + diff_v_y[0, 0, 0]
    
    return diff_u, diff_v

Stencils definition and compilation
--

In [12]:
dtypes = {"dtype": dtype}
externals = {
    "absolute_value": absolute_value,
    "advection_x": advection_x,
    "advection_y": advection_y,
    "advection": advection,
    "diffusion_x": diffusion_x,
    "diffusion_y": diffusion_y,
    "diffusion": diffusion
}


def rk_stage_def(
    in_u_now: gt.gtscript.Field["dtype"],
    in_v_now: gt.gtscript.Field["dtype"],
    in_u_tmp: gt.gtscript.Field["dtype"],
    in_v_tmp: gt.gtscript.Field["dtype"],
    out_u: gt.gtscript.Field["dtype"],
    out_v: gt.gtscript.Field["dtype"],
    *,
    dt: float,
    dx: float,
    dy: float,
    mu: float
):
    with computation(PARALLEL), interval(...):
        adv_u, adv_v = advection(dx=dx, dy=dy, u=in_u_tmp, v=in_v_tmp)
        diff_u, diff_v = diffusion(dx=dx, dy=dy, u=in_u_tmp, v=in_v_tmp)
        out_u = in_u_now + dt * (- adv_u + mu * diff_u)
        out_v = in_v_now + dt * (- adv_v + mu * diff_v)

    
%time rk_stage = gt.gtscript.stencil(backend, rk_stage_def, dtypes=dtypes, externals=externals, rebuild=False)    
    

def copy_def(in_phi: gt.gtscript.Field["dtype"], out_phi: gt.gtscript.Field["dtype"]):
    with computation(PARALLEL), interval(...):
        out_phi = in_phi
    
    
%time copy = gt.gtscript.stencil(backend, copy_def, dtypes=dtypes, externals=externals, rebuild=False)

CPU times: user 7.12 ms, sys: 1.46 ms, total: 8.58 ms
Wall time: 7.61 ms
CPU times: user 4.99 ms, sys: 1.01 ms, total: 6 ms
Wall time: 5.93 ms


Initial and boundary conditions
--

In [4]:
def solution_factory(t, x, y, slice_x=None, slice_y=None):
    nx, ny = x.shape[0], y.shape[0]
    
    slice_x = slice_x or slice(0, nx)
    slice_y = slice_y or slice(0, ny)
    
    mi = slice_x.stop - slice_x.start
    mj = slice_y.stop - slice_y.start
    
    x2d = np.tile(x[slice_x, np.newaxis, np.newaxis], (1, mj, 1))
    y2d = np.tile(y[np.newaxis, slice_y, np.newaxis], (mi, 1, 1))
        
    if use_case == "zhao":
        u = - 4. * mu * np.pi * np.exp(- 5. * np.pi**2 * mu * t) * \
            np.cos(2. * np.pi * x2d) * np.sin(np.pi * y2d) / \
            (2. + np.exp(- 5. * np.pi**2 * mu * t) * np.sin(2. * np.pi * x2d) * np.sin(np.pi * y2d))
        v = - 2. * mu * np.pi * np.exp(- 5.0 * np.pi**2 * mu * t) * \
            np.sin(2. * np.pi * x2d) * np.cos(np.pi * y2d) / \
            (2. + np.exp(- 5. * np.pi**2 * mu * t) * np.sin(2. * np.pi * x2d) * np.sin(np.pi * y2d))
    elif use_case == "hopf_cole":
        u = .75 - 1. / (4. * (1. + np.exp(- t - 4.*x2d + 4.*y2d) / (32.*mu)))
        v = .75 + 1. / (4. * (1. + np.exp(- t - 4.*x2d + 4.*y2d) / (32.*mu)))
    else:
        raise NotImplementedError()
    
    return u, v


def set_initial_solution(x, y, u, v):
    u[...], v[...] = solution_factory(0., x, y)
    
    
def enforce_boundary_conditions(t, x, y, u, v):
    nx, ny = x.shape[0], y.shape[0]
    
    slice_x, slice_y = slice(0, 3), slice(0, ny)
    u[slice_x, slice_y], v[slice_x, slice_y] = solution_factory(t, x, y, slice_x, slice_y)
    
    slice_x, slice_y = slice(nx-3, nx), slice(0, ny)
    u[slice_x, slice_y], v[slice_x, slice_y] = solution_factory(t, x, y, slice_x, slice_y)
    
    slice_x, slice_y = slice(3, nx-3), slice(0, 3)
    u[slice_x, slice_y], v[slice_x, slice_y] = solution_factory(t, x, y, slice_x, slice_y)
    
    slice_x, slice_y = slice(3, nx-3), slice(ny-3, ny)
    u[slice_x, slice_y], v[slice_x, slice_y] = solution_factory(t, x, y, slice_x, slice_y)

Time marching
--

In [10]:
x = np.linspace(0., 1., nx)
dx = 1. / (nx - 1)
y = np.linspace(0., 1., ny)
dy = 1. / (ny - 1)

u_now = gt.storage.zeros(backend, halo, (nx, ny, 1), dtype)
v_now = gt.storage.zeros(backend, halo, (nx, ny, 1), dtype)
u_new = gt.storage.zeros(backend, halo, (nx, ny, 1), dtype)
v_new = gt.storage.zeros(backend, halo, (nx, ny, 1), dtype)

set_initial_solution(x, y, u_new, v_new)

rk_fraction = (1./3., .5, 1.)

t = 0.

# start timing
wall_time_start = time.time()

for i in range(niter):
    copy(in_phi=u_new, out_phi=u_now, origin=(0, 0, 0), domain=(nx, ny, 1))
    copy(in_phi=v_new, out_phi=v_now, origin=(0, 0, 0), domain=(nx, ny, 1))
        
    for k in range(3):
        dt = rk_fraction[k] * timestep
         
        rk_stage(
            in_u_now=u_now, in_v_now=v_now, in_u_tmp=u_new, in_v_tmp=v_new,
            out_u=u_new, out_v=v_new, dt=dt, dx=dx, dy=dy, mu=mu,
            origin=(3, 3, 0), domain=(nx-6, ny-6, 1)
        )
        
        enforce_boundary_conditions(t + dt, x, y, u_new, v_new)
        
    t += timestep
    
    if print_period > 0 and (i+1 % print_period == 0 or i+1 == niter):
        u_ex, v_ex = solution_factory(t, x, y)
        err_u = np.linalg.norm(u_new[3:-3, 3:-3] - u_ex[3:-3, 3:-3]) * np.sqrt(dx * dy)
        err_v = np.linalg.norm(v_new[3:-3, 3:-3] - v_ex[3:-3, 3:-3]) * np.sqrt(dx * dy)
        print(
            "Iteration {:6d}: ||u - uex|| = {:8.4E} m/s, ||v - vex|| = {:8.4E} m/s".format(
                i + 1, err_u, err_v
            )
        )
        
# stop timing
wall_time = time.time() - wall_time_start

print(f"Total wall-clock time for {niter} iterations: {wall_time} s")
print(f"Wall-clock time per iteration: {wall_time/niter} s")

Total wall-clock time for 5 iterations: 1.047226905822754 s
Wall-clock time per iteration: 0.20944538116455078 s


References
--

Baldauf, M. (2008). Stability analysis for linear discretisations of the advection equation with Runge–Kutta time integration. *J. Comp. Phys.*, 227:6638-6659.

Wicker, L. J., and Skamarock, W. C. (2002). Time-splitting methods for elastic models using forward time schemes. *Mon. Wea. Rev.*, 130:2088-2097.

Zhao, G., Yu, X., and Zhang, R. (2011). The new numerical method for solving the system of two-dimensional Burgers’ equations. *Comput. Math. Appl.*, 62:3279-3291.

Zhu, H., Shu, H., and Ding, M. (2010). Numerical solutions of two-dimensional Burgers’ equations by discrete Adomian decomposition method. *Comput. Math. Appl.*, 60:840-848.